# ===============================


# o objetivo do notebook é aplicar a modelagem escolhida nos dados de clima resgatados através da API


# ===============================

# ===============================


# 1 . Importando as bibliotecas básicas


# ===============================

In [44]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import time
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import requests
import json
import datetime


In [60]:
hoje_ano = datetime.datetime.today().year
hoje_mes = datetime.datetime.today().month
hoje_dia = datetime.datetime.today().day

In [61]:
ontem_ano = (datetime.datetime.today() - datetime.timedelta(days=1)).year
ontem_mes = (datetime.datetime.today() - datetime.timedelta(days=1)).month
ontem_dia = (datetime.datetime.today() - datetime.timedelta(days=1)).day

In [62]:
response_ontem = requests.get("https://apitempo.inmet.gov.br/estacao/dados/"+str(ontem_ano)+"-"+str(ontem_mes)+"-"+str(ontem_dia))
response_hoje = requests.get("https://apitempo.inmet.gov.br/estacao/dados/"+str(hoje_ano)+"-"+str(hoje_mes)+"-"+str(hoje_dia))

In [ ]:
df_ontem = pd.read_json(response_ontem.content)
df_hoje = pd.read_json(response_hoje.content)

In [64]:
abt_final4 = pd.read_csv(r'E:\Users\quadr\Documents\MBA\Trabalhos\TCC\Dengue\Bases\Bases Tratadas//abt_final4.csv')

FileNotFoundError: [Errno 2] File b'E:\\Users\\quadr\\Documents\\MBA\\Trabalhos\\TCC\\Dengue\\Bases\\Bases Tratadas//abt_final4.csv' does not exist: b'E:\\Users\\quadr\\Documents\\MBA\\Trabalhos\\TCC\\Dengue\\Bases\\Bases Tratadas//abt_final4.csv'

In [16]:
json.loads(response.content)[0]

{'VEN_DIR': '130',
 'DT_MEDICAO': '2020-05-02',
 'DC_NOME': 'BRASILIA',
 'CHUVA': '0',
 'PRE_INS': '890.9',
 'VL_LATITUDE': '-15.789343',
 'PRE_MIN': '890.5',
 'UMD_MAX': '75',
 'PRE_MAX': '890.9',
 'VEN_VEL': '1.6',
 'UF': 'DF',
 'PTO_MIN': '13.4',
 'TEM_MAX': '19.7',
 'RAD_GLO': '-3.54',
 'PTO_INS': '13.4',
 'VEN_RAJ': '3',
 'TEM_INS': '19.2',
 'UMD_INS': '69',
 'CD_ESTACAO': 'A001',
 'TEM_MIN': '18.5',
 'VL_LONGITUDE': '-47.925756',
 'HR_MEDICAO': '0000',
 'UMD_MIN': '68',
 'PTO_MAX': '14.2'}

# ===============================


# 2 . Voltando para a modelagem


# ===============================

In [2]:
abt_final4 = pd.read_csv(r'E:\Users\quadr\Documents\MBA\Trabalhos\TCC\Dengue\Bases\Bases Tratadas//abt_final4.csv')

In [3]:
abt_final5 = abt_final4.drop('Data',axis = 1)

In [4]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaled_abt = pd.DataFrame(scaler.fit_transform(abt_final5), index=abt_final5.index, columns=abt_final5.columns)


C:\Users\quadr\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:625: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
C:\Users\quadr\Anaconda3\lib\site-packages\sklearn\base.py:462: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)


In [5]:
#Tratando caracteres especiais nos nomes de colunas


scaled_abt.columns = ["".join (c if c.isalnum() else "_" for c in str(x)) for x in scaled_abt.columns]



import re
scaled_abt = scaled_abt.rename(columns = lambda x:re.sub('[^A-Za-z0-9_]+', '', x))

In [6]:
x = scaled_abt.drop(['NU_NOTIFIC'], axis =1)
y = scaled_abt.NU_NOTIFIC

##  Começando a Modelagem

In [7]:
from sklearn.metrics import mean_absolute_error as mae,mean_squared_error as mse
from sklearn.model_selection import train_test_split

In [8]:
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size = 0.3, random_state = 42)

## Regressão Linear

In [9]:
from sklearn.linear_model import LinearRegression

lr = LinearRegression()

lr_model = lr.fit(x_train, y_train)
lr_pred = lr_model.predict(x_test)
lr_pred_train = lr_model.predict(x_train)


# Verificando a performance do modelo

print('treino')
print('O Score do Modelo na base de treino é : ',lr_model.score(x_train,y_train))
print('O MAE do Modelo  é : ', mae(y_train,lr_pred_train))
print('O MSRE do Modelo  é : ', np.sqrt(mse(y_train,lr_pred_train)))



print('O Score do Modelo de Regressão Linear na base de teste é : ', lr.score(x_test,y_test))
print('O MAE do Modelo de Regressão Linear é : ', mae(y_test,lr_pred))
print('O MSRE do Modelo de Regressão Linear é : ', np.sqrt(mse(y_test,lr_pred)))

treino
O Score do Modelo na base de treino é :  0.04243864582065282
O MAE do Modelo  é :  0.30328138661084625
O MSRE do Modelo  é :  0.9752572265853496
O Score do Modelo de Regressão Linear na base de teste é :  0.04738105583432839
O MAE do Modelo de Regressão Linear é :  0.3022868897346913
O MSRE do Modelo de Regressão Linear é :  0.9836433699612683


## Árvore de Decisão

In [10]:
from sklearn.tree import DecisionTreeRegressor

tree = DecisionTreeRegressor(random_state=42)
tree_model = tree.fit(x_train,y_train)
tree_pred = tree_model.predict(x_test)
tree_pred_train = tree_model.predict(x_train)


# Verificando a performance do modelo

print('treino')
print('O Score do Modelo na base de treino é : ',tree_model.score(x_train,y_train))
print('O MAE do Modelo  é : ', mae(y_train,tree_pred_train))
print('O MSRE do Modelo  é : ', np.sqrt(mse(y_train,tree_pred_train)))

print('O Score do Modelo de Árvore de Decisão na base de teste é : ',tree_model.score(x_test,y_test))
print('O MAE do Modelo de Árvore de Decisão é : ', mae(y_test,tree_pred))
print('O MSRE do Modelo de Árvore de Decisão é : ', np.sqrt(mse(y_test,tree_pred)))

treino
O Score do Modelo na base de treino é :  1.0
O MAE do Modelo  é :  8.744503923399253e-18
O MSRE do Modelo  é :  2.9671535272569776e-17
O Score do Modelo de Árvore de Decisão na base de teste é :  0.46821091972143764
O MAE do Modelo de Árvore de Decisão é :  0.1375560065155145
O MSRE do Modelo de Árvore de Decisão é :  0.7349330359539542


In [11]:
abt_final5.NU_NOTIFIC.describe()

count    120848.000000
mean         26.676006
std         119.888775
min           1.000000
25%           2.000000
50%           5.000000
75%          16.000000
max        4017.000000
Name: NU_NOTIFIC, dtype: float64

## ===============================


# 5 . Modelagem - Inteligência Artificial


# ===============================

### Ainda sem realizar nenhum tipo de tratamento adicional, vamos aplicar alguns modelos de inteligência artificial, para comparar o resultado e definir quais os modelos seguiriam para uma etapa de otimização

## Random Forest

In [25]:
from sklearn.ensemble import RandomForestRegressor

rf = RandomForestRegressor(random_state=42)
rf_model = rf.fit(x_train,y_train)
rf_pred_train = rf.predict(x_train)
rf_pred = rf_model.predict(x_test)


# Verificando a performance do modelo

print('treino')
print('O Score do Modelo na base de treino é : ',rf_model.score(x_train,y_train))
print('O MAE do Modelo  é : ', mae(y_train,rf_pred_train))
print('O MSRE do Modelo  é : ', np.sqrt(mse(y_train,rf_pred_train)))

print('teste')
print('O Score do Modelo  na base de teste é : ',rf_model.score(x_test,y_test))
print('O MAE do Modelo  é : ', mae(y_test,rf_pred))
print('O MSRE do Modelo  é : ', np.sqrt(mse(y_test,rf_pred)))

C:\Users\quadr\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


treino
O Score do Modelo na base de treino é :  0.9499331382784686
O MAE do Modelo  é :  0.04474466688881117
O MSRE do Modelo  é :  0.22300318043156148
teste
O Score do Modelo  na base de teste é :  0.7148550122901596
O MAE do Modelo  é :  0.11332094408850048
O MSRE do Modelo  é :  0.5381594032423808


## XGBoost

In [11]:
from xgboost import XGBRegressor

xgb = XGBRegressor(random_state=42)
xgb_model = xgb.fit(x_train,y_train)
xgb_pred = xgb.predict(x_test)
xgb_pred_train = xgb.predict(x_train)


# Verificando a performance do modelo

print('treino')
print('O Score do Modelo na base de treino é : ',xgb_model.score(x_train,y_train))
print('O MAE do Modelo  é : ', mae(y_train,xgb_pred_train))
print('O MSRE do Modelo  é : ', np.sqrt(mse(y_train,xgb_pred_train)))

print('teste')
print('O Score do Modelo  na base de teste é : ',xgb_model.score(x_test,y_test))
print('O MAE do Modelo  é : ', mae(y_test,xgb_pred))
print('O MSRE do Modelo  é : ', np.sqrt(mse(y_test,xgb_pred)))

C:\Users\quadr\Anaconda3\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


[21:51:59] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
treino
O Score do Modelo na base de treino é :  0.6155852522951435
O MAE do Modelo  é :  0.1963865895971043
O MSRE do Modelo  é :  0.6179251793056751
teste
O Score do Modelo  na base de teste é :  0.585665365925512
O MAE do Modelo  é :  0.19961840968200215
O MSRE do Modelo  é :  0.648714612685285


## LGBM

In [19]:
from lightgbm import LGBMRegressor

lgbm = LGBMRegressor(random_state=42)
lgbm_model = lgbm.fit(x_train,y_train)
lgbm_pred = lgbm_model.predict(x_test)
lgbm_pred_train = lgbm_model.predict(x_train)


# Verificando a performance do modelo

print('treino')
print('O Score do Modelo na base de treino é : ',lgbm_model.score(x_train,y_train))
print('O MAE do Modelo  é : ', mae(y_train,lgbm_pred_train))
print('O MSRE do Modelo  é : ', np.sqrt(mse(y_train,lgbm_pred_train)))

print('teste')
print('O Score do Modelo  na base de teste é : ',lgbm_model.score(x_test,y_test))
print('O MAE do Modelo  é : ', mae(y_test,lgbm_pred))
print('O MSRE do Modelo  é : ', np.sqrt(mse(y_test,lgbm_pred)))

treino
O Score do Modelo na base de treino é :  0.8632221671128908
O MAE do Modelo  é :  0.1347522494221758
O MSRE do Modelo  é :  0.36859015510151444
teste
O Score do Modelo  na base de teste é :  0.6957820671488806
O MAE do Modelo  é :  0.15323523728574012
O MSRE do Modelo  é :  0.5558664566090609


## ===============================


# 6 . Otimização dos Modelos


# ===============================

### Os modelos com melhor desempenho foram o Random Forest e o LightGBM


### Começando com o Cross Validation

In [16]:
rf_model.get_params

<bound method BaseEstimator.get_params of RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=None,
           oob_score=False, random_state=42, verbose=0, warm_start=False)>

In [22]:
lgbm_model.get_params

<bound method LGBMModel.get_params of LGBMRegressor(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
       importance_type='split', learning_rate=0.1, max_depth=-1,
       min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
       n_estimators=100, n_jobs=-1, num_leaves=31, objective=None,
       random_state=42, reg_alpha=0.0, reg_lambda=0.0, silent=True,
       subsample=1.0, subsample_for_bin=200000, subsample_freq=0)>

In [23]:
# Primeiro para o Random Forest

inicio = time.time()

from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV


rf = RandomForestRegressor(random_state=42)
rf_model = rf.fit(x_train,y_train)
rf_pred = rf.predict(x_test)

parameters = {'max_depth':[30,50,100], 'criterion':['mse'], 'n_estimators':[50,100,150]}


grid_rf = GridSearchCV(estimator= rf, param_grid=parameters ,verbose=1)
grid_rf_model = grid_rf.fit(x_train,y_train)
grid_rf_pred = grid_rf_model.predict(x_test)
grid_rf_pred_train = grid_rf_model.predict(x_train)




# Verificando a performance do modelo


print('treino')
print('O Score do Modelo com Cross Validation é : ',grid_rf_model.score(x_train,y_train))
print('O MAE do com Cross Validation é : ', mae(y_train,grid_rf_pred_train))
print('O MSRE do Modelo com Cross Validation  é : ', np.sqrt(mse(y_train,grid_rf_pred_train)))


print('teste')
print('O Score do Modelo com Cross Validation é : ',grid_rf_model.score(x_test,y_test))
print('O MAE do com Cross Validation é : ', mae(y_test,grid_rf_pred))
print('O MSRE do Modelo com Cross Validation  é : ', np.sqrt(mse(y_test,grid_rf_pred)))



grid_rf_model.get_params

fim = time.time()

print('Tempo Gasto = ', (fim - inicio))

C:\Users\quadr\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\quadr\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Fitting 3 folds for each of 9 candidates, totalling 27 fits


[Parallel(n_jobs=1)]: Done  27 out of  27 | elapsed: 139.4min finished


treino
O Score do Modelo com Cross Validation é :  0.9645505811223444
O MAE do com Cross Validation é :  0.04015076076662767
O MSRE do Modelo com Cross Validation  é :  0.1876464843313224
teste
O Score do Modelo com Cross Validation é :  0.7384898904247086
O MAE do com Cross Validation é :  0.10746659464844052
O MSRE do Modelo com Cross Validation  é :  0.5153737606776986
Tempo Gasto =  8911.893840789795


In [24]:
# Para o LGBM

inicio = time.time()

from lightgbm import LGBMRegressor
from sklearn.model_selection import GridSearchCV


lgbm = LGBMRegressor(random_state=42)
lgbm_model = lgbm.fit(x_train,y_train)
lgbm_pred = lgbm_model.predict(x_test)


parameters = {'max_depth':[30,50,100],'boosting_type':['gbdt','dart','goss'] ,'n_estimators':[50,100,150]}


grid_lgbm = GridSearchCV(estimator= lgbm, param_grid=parameters ,verbose=1)
grid_lgbm_model = grid_lgbm.fit(x_train,y_train)
grid_lgbm_pred = grid_lgbm_model.predict(x_test)
grid_lgbm_pred_train = grid_lgbm_model.predict(x_train)




# Verificando a performance do modelo

print('treino')
print('O Score do Modelo com Cross Validation é : ',grid_lgbm_model.score(x_train,y_train))
print('O MAE do com Cross Validation é : ', mae(y_train,grid_lgbm_pred_train))
print('O MSRE do Modelo com Cross Validation  é : ', np.sqrt(mse(y_train,grid_lgbm_pred_train)))


print('teste')
print('O Score do Modelo com Cross Validation é : ',grid_lgbm_model.score(x_test,y_test))
print('O MAE do com Cross Validation é : ', mae(y_test,grid_lgbm_pred))
print('O MSRE do Modelo com Cross Validation  é : ', np.sqrt(mse(y_test,grid_lgbm_pred)))

grid_lgbm_model.get_params

fim = time.time()

print('Tempo Gasto = ', (fim - inicio))

C:\Users\quadr\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Fitting 3 folds for each of 27 candidates, totalling 81 fits


[Parallel(n_jobs=1)]: Done  81 out of  81 | elapsed:  2.6min finished


treino
O Score do Modelo com Cross Validation é :  0.8855911986866912
O MAE do com Cross Validation é :  0.12491610272942556
O MSRE do Modelo com Cross Validation  é :  0.3371053070530676
teste
O Score do Modelo com Cross Validation é :  0.7040374583570184
O MAE do com Cross Validation é :  0.14768985091877188
O MSRE do Modelo com Cross Validation  é :  0.5482724658712087
Tempo Gasto =  162.2355272769928


## Fazendo feature selection

In [22]:
from sklearn.feature_selection import SelectPercentile, f_regression

for i in range(2,12,2):
    
    print(str(i*10)+'%')
    print('-------------')
    x_new = SelectPercentile(f_regression,percentile=i*10).fit_transform(x,y)
    
    x_train,x_test,y_train,y_test = train_test_split(x_new,y,test_size = 0.3, random_state = 42)
    
    from lightgbm import LGBMRegressor

    lgbm = LGBMRegressor(random_state=42)
    lgbm_model = lgbm.fit(x_train,y_train)
    lgbm_pred = lgbm_model.predict(x_test)
    lgbm_pred_train = lgbm_model.predict(x_train)


    # Verificando a performance do modelo

    print('treino')
    print('O Score do Modelo na base de treino é : ',lgbm_model.score(x_train,y_train))
    print('O MAE do Modelo  é : ', mae(y_train,lgbm_pred_train))
    print('O MSRE do Modelo  é : ', np.sqrt(mse(y_train,lgbm_pred_train)))

    print('teste')
    print('O Score do Modelo  na base de teste é : ',lgbm_model.score(x_test,y_test))
    print('O MAE do Modelo  é : ', mae(y_test,lgbm_pred))
    print('O MSRE do Modelo  é : ', np.sqrt(mse(y_test,lgbm_pred))) 
    
    print('-----------')



20%
-------------
treino
O Score do Modelo na base de treino é :  0.7172510764954623
O MAE do Modelo  é :  0.1648865253210031
O MSRE do Modelo  é :  0.5299517761351455
teste
O Score do Modelo  na base de teste é :  0.6030392402520334
O MAE do Modelo  é :  0.17543289887470395
O MSRE do Modelo  é :  0.6349680177663659
-----------
40%
-------------
treino
O Score do Modelo na base de treino é :  0.8189368231889995
O MAE do Modelo  é :  0.14988703985944102
O MSRE do Modelo  é :  0.4240830746695275
teste
O Score do Modelo  na base de teste é :  0.6682234536550674
O MAE do Modelo  é :  0.1651071234041812
O MSRE do Modelo  é :  0.5804982343560475
-----------
60%
-------------
treino
O Score do Modelo na base de treino é :  0.8458201773197234
O MAE do Modelo  é :  0.14198927721363516
O MSRE do Modelo  é :  0.3913358569948988
teste
O Score do Modelo  na base de teste é :  0.6880204649770452
O MAE do Modelo  é :  0.15832842177915657
O MSRE do Modelo  é :  0.5629127886112127
-----------
80%
-----

### Não tivemos mudanças positivas com número de features menor, manteremos 100%